In [90]:
import cv2
import numpy as np


In [91]:
### Loading Yolo

In [92]:
net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
classes=[]
with open("coco.names","r") as f:
    classes=[line.strip() for line in f.readlines()]

print(len(classes))

80


In [93]:
layers_names=net.getLayerNames()

In [94]:
output_layers=[layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
output_layers
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [95]:
net.getUnconnectedOutLayers()

array([[200],
       [227],
       [254]], dtype=int32)

In [96]:
img=cv2.imread("room_ser.jpg")
img=cv2.resize(img,None,fx=0.4,fy=0.4)
height,width,channels=img.shape


In [97]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)


In [98]:
net.setInput(blob)
outs=net.forward(output_layers)

In [99]:
print(len(outs))

3


In [100]:
class_ids=[]
confidences=[]
boxes=[]
for out in outs:
    for detection in out:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        
        if confidence > 0.5:
            ## Object detected
            center_x=int(detection[0]*width)
            center_y=int(detection[1]*height)
            w=int(detection[2]*width)
            h=int(detection[3]*height)
            
            x=int(center_x- w/2)
            y=int(center_y- h/2)
            ## cv2.circle(img,(center_x,center_y),10,(0,0,255),1)
            ## Rectangle coordinates
            ## cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
print(indexes)
##number_of_boxes=len(boxes)
font=cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h=boxes[i]
        label=str(classes[class_ids[i]])
        color=colors[i]
        print(label)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(img,label,(x,y),font,3,color,3)
cv2.imshow("Img",img)
cv2.waitKey(0)
        
        

[[ 2]
 [ 5]
 [ 8]
 [12]]
laptop
chair
tvmonitor
book


-1